# Автоматическая обработка текстов
## Домашнее задание 2 [10 баллов]
В этом домашнем задании вам потребуется
* проанализировать коллекцию текстов (новостных сообщений) и построить ее тематическую модель.

# Тематическое моделирование.

### Загружаем данные
Для тех, у кого windows (скачайте и распакуйте вручную), выполнять следующие строчки нет необходимости.

In [ ]:
!wget https://github.com/yutkin/lenta.ru-news-dataset/raw/master/data/lenta_data.tar.gz.aa

In [ ]:
!tar -xvf lenta_data.tar.gz.aa

### Подготовка и первичный анализ данных [3 балла]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_full = pd.read_csv("lenta_data.csv", error_bad_lines=False, encoding="utf-8", engine="python")

In [ ]:
topics = df_full.topic.factorize()[1]

In [ ]:
df_full.topic = df_full.topic.factorize()[0]

In [ ]:
df_full.head()

In [ ]:
df_full.count()

#### Посмотрите распределение тем в датасете

In [ ]:
len(topics), topics

Постпройте гистограмму тем (df_full.topic)

In [ ]:
<your code here>

In [ ]:
plt.figure(figsize=(15, 7))
df_full.topic.hist(bins=18)

Что можно сказать о таком распределении

<your answer here>

<your answer here>

Если для вашего железа данных много (а скорее всего так и есть) сделайте подвыборку из данных (отбросьте строки для классов, которых больше всего).
Для вас уже написан код, который это делает.

In [ ]:
import numpy as np


def balanced_sample_maker(X, y, sample_size, random_seed=None):
    """ return a balanced data set by sampling all classes with sample_size 
        current version is developed on assumption that the positive
        class is the minority.

    Parameters:
    ===========
    X: {numpy.ndarrray}
    y: {numpy.ndarray}
    """
    uniq_levels = np.unique(y)
    uniq_counts = {level: sum(y == level) for level in uniq_levels}

    if not random_seed is None:
        np.random.seed(random_seed)

    # find observation index of each class levels
    groupby_levels = {}
    for ii, level in enumerate(uniq_levels):
        obs_idx = [idx for idx, val in enumerate(y) if val == level]
        groupby_levels[level] = obs_idx
    # oversampling on observations of each label
    balanced_copy_idx = []
    for gb_level, gb_idx in groupby_levels.items():
        over_sample_idx = np.random.choice(gb_idx, size=min(sample_size, len(groupby_levels[gb_level])), replace=True).tolist()
        balanced_copy_idx+=over_sample_idx
    np.random.shuffle(balanced_copy_idx)

    return X[balanced_copy_idx], y[balanced_copy_idx], balanced_copy_idx

In [ ]:
X, y, balanced_copy_idx = balanced_sample_maker(np.array(df_full.text), df_full.topic, 500)

In [ ]:
df = df_full.iloc[balanced_copy_idx]

In [ ]:
len(df), len(df.topic.unique())

In [ ]:
df.head()

Постпройте гистограмму тем (df.topic)

In [ ]:
plt.figure(figsize=(15, 7))
df.topic.hist(bins=18)

Правильно ли мы сделали подвыборку?

<your answer here>

#### Удалите из текста лишние символы/стоп-слова, лемматизируйте
(Если считаете это необходимым)

In [ ]:
<your code here>

#### Постройте облако слов

In [ ]:
from nltk import FreqDist


lemmata = []
for index, row in df.iterrows():
    lemmata += row['text'].split()

fd = FreqDist(lemmata)

for i in fd.most_common(10):
    print(i)

In [ ]:
from wordcloud import *


word_freq = [i for i in fd.most_common(100)]
wd = WordCloud(background_color = 'white')
wd.generate_from_frequencies(dict(word_freq))
plt.figure(figsize=(15, 7))
plt.imshow(wd, interpolation = 'bilinear')
plt.axis('off')
plt.show()

Есть ли необходимость удалять какие-то слова из текста?

<your answer here>

### Визуализация данных [1 балл]
Используйте T-SNE, чтобы посмотреть на данные в двумерном виде.

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer


vectors = <your code here>
X_reduced = <your code here>
X_embedded = <your code here>

In [ ]:
vis_df = pd.DataFrame({'X': X_embedded[:, 0], 'Y': X_embedded[:, 1], 'topic' : df.topic.apply(lambda x: topics[x])})
sns.FacetGrid(vis_df, hue="topic", size=10).map(plt.scatter, "X", "Y").add_legend()

Можно ли сделать какой-то вывод по этому графику?

<your answer here>

<your answer here>

### Создайте корпус и обучите тематическую модель [2 балла]
Выделите термины из текста и создайте корпус для gensim или другой библиотеки.

In [ ]:
<your code here>

### Интерпретация результатов [2 балла]
Посмотрите на результаты с помощью gensimvis

In [ ]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

Проинтерпретируйте результаты и попытайтесь соотнести темы с topics.

<your answer here>

### Оцените качество работы модели [1 балл]
Вычислите перплексию для модели.

In [ ]:
<your code here>

### Применение модели [1 балл]
Выберите несколько текстов из коллекции и найдите распределение тем для них.

In [ ]:
<your code here>

## Дополнительное задание * [5 баллов]
Проведите анализ зависимости качества от терминов, которые используются в модели

In [ ]:
<your code here>

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  конец курса. 

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.


Сдача отчетов осуществляется по email: login-const@mail.ru, заголовок письма – NLP-HSE-HW2-ИмяФамилия. 
